In [1]:
from diffusers import DiffusionPipeline
import torch
from PIL import Image
import os
import argparse
import yaml
import pickle
import json
import shutil
import sys
import uuid
from datetime import datetime

sys.path.append("./diffusers")

/workspace/oa_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspace/Diploma/OneActor/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
def cat_images(images):
    widths, heights = zip(*(i.size for i in images))

    total_width = sum(widths)
    max_height = max(heights)

    new_image = Image.new('RGB', (total_width, max_height))

    x_offset = 0
    for img in images:
        new_image.paste(img, (x_offset, 0))
        x_offset += img.width
    return new_image

def decode_latent(latent, pipeline):
    with torch.no_grad():
        pipeline.upcast_vae()
        latent = latent.to(next(iter(pipeline.vae.post_quant_conv.parameters())).dtype)
        image = pipeline.vae.decode(latent / pipeline.vae.config.scaling_factor, return_dict=False)[0]
        image = pipeline.image_processor.postprocess(image, output_type='pil')[0]
    return image

def decode_and_cat(latent_list, pipeline):
    images = []
    for i in latent_list:
        images.append(decode_latent(i, pipeline))
    image = cat_images(images)
    return image


In [5]:
    with open("PATH.json","r") as f:
        ENV_CONFIGS = json.load(f)
    with open("./config/gen_tune_inference_man.yaml", "r") as f:
        config = yaml.safe_load(f)

In [6]:
    device = config['device']
    pipeline = DiffusionPipeline.from_pretrained(ENV_CONFIGS['paths']['sdxl_path']).to(device)
    prompt = config['target_prompt'] + " " + config['add_target_prompt']
    print(f"Target prompt: {prompt}")
    guidance_scale = config['guidance_scale']
    steps = config['steps']
    generator = torch.manual_seed(config['g_seed'])

/workspace/oa_venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/workspace/Diploma/OneActor/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/workspace/oa_venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  8.65it/s]


Target prompt: A rugger adventurer with tousled hair, comic book stile 


In [7]:
    image, target_xt_list_, target_prompt_embeds, target_mid_ = pipeline(prompt, neg_prompt=config['target_neg_prompt'], 
                                                    num_inference_steps=steps, guidance_scale=guidance_scale, generator=generator,
                                                    oneactor_save=True)
    # save the target image
    target_image = image.images[0]


100%|██████████| 30/30 [00:24<00:00,  1.24it/s]


In [14]:
    base_image = []
    mid_last_base = []
    mid_last_base_long = []
    if config['gen_base'] > 0:
        num_base = 3
        mid_last_base = []
        for i in range(num_base):
            image, xt_list_, prompt_embeds, mid_ = pipeline(prompt, neg_prompt=config['target_neg_prompt'],
                                                            num_inference_steps=steps, guidance_scale=guidance_scale, generator=generator,
                                                            oneactor_save=True)
            base_image.append(image.images[0])
            mid_last_base.append(mid_[-1].cpu())
            mid_last_base_long.append([_.cpu() for _ in mid_])

100%|██████████| 30/30 [00:24<00:00,  1.21it/s]
